In [ ]:
### SETTING UP DIRECTORIES

# import packages
import pandas as pd
import os
import numpy as np

# set working directory for da_data_repo -- replace the
os.chdir("/Users/.../da_data_repo")

# location folders
data_in = "working-from-home/raw/"
data_out = "working-from-home/clean/"

In [ ]:
#load dta

df1 = pd.read_stata(data_in+'quit_data.dta')

In [ ]:
df1.rename(
    columns={
        "expgroup": "treatment",
        "men":"male"
    },
    inplace=True,
)

In [ ]:
df1.drop(
    columns=["perform10_expgroup", "perform11_expgroup"],
    inplace=True,
)

In [ ]:
df2 = pd.read_stata(data_in+'tc_comparison.dta')

In [ ]:
df2.rename(
    columns={
        "expgroup": "treatment",
        "men":"male"
    },
    inplace=True,
)

In [ ]:
cols_to_use = df1.columns.difference(df2.columns).tolist()

In [ ]:
cols_to_use.append('personid')

In [ ]:
df_merged = pd.merge(df1[cols_to_use], df2, on="personid", how="inner")

In [ ]:
cols_to_order = ['personid','treatment','quitjob','perform10','perform11']

In [ ]:
new_columns = cols_to_order + (df_merged.columns.drop(cols_to_order).tolist())
df_merged = df_merged[new_columns]

In [ ]:
#load dta

df3 = pd.read_stata(data_in+'performance_during_exper.dta')

In [ ]:
df3=df3.loc[(df3.expgroup == 0) |(df3.expgroup == 1)].reset_index()

In [ ]:
df3['experiment_time'] = df3.experiment_treatment + df3.experiment_control

In [ ]:
df3['phonecalls']=df3.phonecallraw/1000

df3.drop(
    columns=["phonecallraw"],
    inplace=True,
)

In [ ]:

df3=df3[['personid', 'year_week', 'experiment_time', 'treatment', 'phonecalls']]

In [ ]:
df3.to_csv(data_out + "wfh_tidy_personweek_p.csv", index=False)

In [ ]:
df3=df3.groupby(['personid', 'experiment_time']).agg({'phonecalls':'sum', 'treatment':'mean'}).reset_index()

In [ ]:

df3= df3.pivot_table(
    index=["personid",'treatment'],
    columns="experiment_time",
    values="phonecalls",
).add_prefix("phonecalls")


In [ ]:
df3=df3.reset_index(level='treatment').rename_axis(None)

In [ ]:
df3.columns.name=None
df3=df3.rename_axis('personid').reset_index()

In [ ]:
df3.rename(
    columns={
        "phonecalls0.0": "phonecalls0",
        "phonecalls1.0":"phonecalls1"
    },
    inplace=True,
)

In [ ]:
df_merged = pd.merge(
    df3[["personid", "phonecalls0", "phonecalls1"]],
    df_merged,
    on="personid",
    how="inner",
)

In [ ]:
df_merged['ordertaker']=[1 if x == 'phone call group' else 0 for x in df_merged.type]

In [ ]:
cols_to_order = ["personid", "treatment", "ordertaker", "type", "quitjob"]
new_columns = cols_to_order + (df_merged.columns.drop(cols_to_order).tolist())
df_merged = df_merged[new_columns]

In [ ]:
df_merged.to_csv(data_out + "wfh_tidy_person_p.csv", index=False)